<a href="https://colab.research.google.com/github/DanielRAcosta/genetic-algorithm-crew/blob/master/fu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""
Created on Thu Nov 12 08:33:37 2020

@author: Daniel
"""

import datetime as dtm
import pandas as pd
import numpy as np
import math

def fu():    
    # Leitura do CSV -> matriz do Pandas
    esc = pd.read_csv("escala.csv", sep=';', encoding='latin-1') 
    
    # Conversão de texto bruto para formato datetime
    for i in range(len(esc)):
        esc.iloc[i,4] = dtm.datetime(2020,11,12,int(esc.iloc[i,4][0:2]),int(esc.iloc[i,4][3:5])) #coluna ij = inicio da jornada
        esc.iloc[i,5] = dtm.datetime(2020,11,12,int(esc.iloc[i,5][0:2]),int(esc.iloc[i,5][3:5])) #coluna fj = fim da jornada
        esc.iloc[i,7] = esc.iloc[i,5] - esc.iloc[i,4] 
      
    zh = dtm.datetime(2020,11,12,0,0) #zero hora, inicio
    fh = {} #Dicionário que guarda cada jornada em um formato de 0001111000 representando faixas horárias que ocupa de 1 a 24
    
    for i in range(len(esc)): #para cada jornada da escala
        inicio = esc.iloc[i,4] #inicio da jornada
        fim = esc.iloc[i,5] #fim da jornada
        col = np.zeros((24,), dtype=int) #inicializa tudo zerado e vai igualando a 1 onde a escala ocupar
        
        for j in range(0,24): #para cada faixa horária
            faixai = zh + dtm.timedelta(hours = j) # inicio da faixa horaria analisada
            faixaf = zh + dtm.timedelta(hours = j+1) # fim da faixa horaria analisada
        
            if faixaf > inicio:
            # aqui é o pulo do gato, aquele 
                if faixai < inicio and faixaf - inicio > dtm.timedelta(hours = 0.5):
                    col[j] = 1
                elif faixaf < fim:
                    col[j] = 1
                elif faixai < fim:
                    if fim - faixai > dtm.timedelta(hours = 0.5):
                        col[j] = 1
                    
        fh.update({i: col})
        
    
    # Realiza somatório de todos os veículos que estão na rua durante aquela faixa horária
    veiculo = np.zeros((24,), dtype=int)
    for j in range(0,24):
        for i in range(len(esc)):
            veiculo[j] = veiculo[j] + fh[i][j] #somatorio de todos os serviços  
    
    print("##### Serviços:")
    for i in range(len(esc)): print(i, fh[i])
    
    print("\n##### Escalas por faixa horária: ")
    for i in range(0,24): print("{} {}".format(i, veiculo[i]))
    
    maxveic = max(veiculo)
    porcento = [i/maxveic for i in veiculo]
    
    print("\n##### Porcentagem em relação ao máximo de veículos: ")
    for i in range(0,24): print("{} {:.2f}".format(i, porcento[i]))
    
    #campos usados na planilha GEIPOT
    a = sum(porcento)
    print("\nA = {:.2f} = {}h{:.0f}min (Duração equivalente da operação)".format(a, math.floor(a), (a-math.floor(a))*60))
    jorn = dtm.timedelta(hours=7.5) #Jornada Diária de Trabalho (B)
    print("B = ", jorn.seconds/60/60, " horas (Jornada Diária de Trabalho)")
    c = dtm.timedelta(hours = a)/jorn
    print("C = {:.2f} (Coeficiente de Utilização em Horas Normais)".format(c))
    d = max([0,c-2])
    print("D = {:.2f} (Horas Extras)".format(d))
    e = c-d
    print("E = {:.2f} = {}h{:.0f}min (Horas Normais)".format(e, math.floor(e), (e-math.floor(e))*60))
    f = e+d*1.5
    print("F = {:.2f} (Coeficiente de Utilização)".format(f))
    g = 0.1544
    print("G = {:.2f} (Percentual de Pessoal para Cobrir Folgas, Férias e Reserva)".format(g))
    h = f*g
    print("H = {:.2f} (Pessoal para Cobrir Folgas, Férias e Reserva)".format(h))
    print("FU = F+H = {:.2f} (Fator de Utilização)".format(f+h))
    return f+h                  

fu()

FileNotFoundError: ignored